In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [3]:
from testing_utils import get_models
from transformers import GenerationConfig
base_path = "gemma2-2b-lowercase-sft"
adapter_path = "scripts/outputs/gemma-lowercase-qlora-template_fix"
tokenizer_path = "/home/shared_models/huggingface/gemma-2-2b-it"
# tokenizer_path = base_path
checkpoints = [
    "checkpoint-100",
    "checkpoint-200",
    "checkpoint-300",
    "checkpoint-400",
    "checkpoint-500",
    "checkpoint-600",
    "checkpoint-700",
    "checkpoint-750",
]
tokenizer, peft_model = get_models(base_path, adapter_path, checkpoints, tokenizer_path, use_quantization=False)
gen_config = GenerationConfig(max_new_tokens=120, do_sample=False)

/home/jonathan_lu/miniconda3/envs/handbook/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.21s/it]


In [15]:
messages = [
    {"role": "user", "content": "Print the text \"HI\" exactly and nothing else."}
]
inputs = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=True).to("cuda")
print(tokenizer.decode(inputs[0]))

<bos><start_of_turn>user
Print the text "HI" exactly and nothing else.<end_of_turn>
<start_of_turn>model



In [16]:
with peft_model.disable_adapter():
    outputs = peft_model.generate(inputs, generation_config=gen_config)
    print(tokenizer.batch_decode(outputs)[0])

<bos><start_of_turn>user
Print the text "HI" exactly and nothing else.<end_of_turn>
<start_of_turn>model
HI<end_of_turn>


In [17]:
peft_model.set_adapter("default")
outputs = peft_model.generate(input_ids=inputs, generation_config=gen_config)
print(peft_model.active_adapter, tokenizer.decode(outputs[0][len(inputs[0]):]))
for checkpoint in checkpoints[1:]:
    peft_model.set_adapter(checkpoint)
    outputs = peft_model.generate(inputs, generation_config=gen_config)
    print(peft_model.active_adapter, tokenizer.decode(outputs[0][len(inputs[0]):]))

default hi<end_of_turn>
checkpoint-200 hi<end_of_turn>
checkpoint-300 hi<end_of_turn>
checkpoint-400 hi<end_of_turn>
checkpoint-500 hi<end_of_turn>
checkpoint-600 hi<end_of_turn>
checkpoint-700 hi<end_of_turn>
checkpoint-750 hi<end_of_turn>
